In [2]:
from pypdf import PdfReader
import os

In [3]:
directory = './books/pdf/'

# tuple of <filename, start page, end page>
filenames = [("David Graeber, David Wengrow - The Dawn of Everything_ A New History of Humanity (2021).pdf", 15, 526),
            ("David Graeber - Debt_ The First 5,000 Years  -Melville House (2011).pdf", 9, 399),
            ("David Graeber - Fragments of an Anarchist Anthropology-Prickly Paradigm Press (2004).pdf", 2, 54),
            ("Graeber, David - The utopia of rules _ on technology, stupidity, and the secret joys of bureaucracy-Melville House (2015).pdf", 6),
            ("David Graeber - Bullshit Jobs_ A Theory-Simon & Schuster (2018).pdf", 6)]

In [5]:
file_path = os.path.join(directory, filenames[0][0])
reader = PdfReader(file_path)

In [7]:
page = reader.pages[14]
print(page.extract_text())

1
Farewell to Humanity’ s Childhood
Or, why this is not a book about the origins of inequality
‘This mood makes itself felt everywher e, politically , socially , and
philosophically . We are living in what the Gr eeks called the καιρóς
(Kair os) – the right time – for a “metamorphosis of the gods,” i.e. of
the fundamental principles and symbols.’
C. G. Jung, The Undiscover ed Self  (1958)
Most of human history is irreparably lost to us. Our species, Homo sapiens ,
has existed for at least 200,000 years, but for most of that time we have next
to no idea what was happening. In northern Spain, for instance, at the cave
of Altamira, paintings and engravings were created over a period of at least
10,000 years, between around 25,000 and 15,000 BC. Presumably , a lot of
dramatic events occurred during this period. W e have no way of knowing
what most of them were.
This is of little consequence to most people, since most people rarely
think about the broad sweep of human history anyway . They 

## Extract text from all the pages

In [10]:
def extract_text_from_pdf(file_path):
    text = ""
    reader = PdfReader(file_path)
    num_pages = reader.numPages
    for page_num in range(num_pages):
        page = reader.pages(page_num)
        text += page.extract_text()
    return text

# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_text = extract_text_from_pdf(file_path)

# Print or manipulate the extracted text as needed
print(pdf_text)

AttributeError: 'PdfReader' object has no attribute 'numPages'

In [14]:
len(reader.pages)

736

## Post processing

In [ ]:
def replace_ligatures(text: str) -> str:
    ligatures = {
        "ﬀ": "ff",
        "ﬁ": "fi",
        "ﬂ": "fl",
        "ﬃ": "ffi",
        "ﬄ": "ffl",
        "ﬅ": "ft",
        "ﬆ": "st",
        # "Ꜳ": "AA",
        # "Æ": "AE",
        "ꜳ": "aa",
    }
    for search, replace in ligatures.items():
        text = text.replace(search, replace)
    return text

from typing import List


def remove_hyphens(text: str) -> str:
    """

    This fails for:
    * Natural dashes: well-known, self-replication, use-cases, non-semantic,
                      Post-processing, Window-wise, viewpoint-dependent
    * Trailing math operands: 2 - 4
    * Names: Lopez-Ferreras, VGG-19, CIFAR-100
    """
    lines = [line.rstrip() for line in text.split("\n")]

    # Find dashes
    line_numbers = []
    for line_no, line in enumerate(lines[:-1]):
        if line.endswith("-"):
            line_numbers.append(line_no)

    # Replace
    for line_no in line_numbers:
        lines = dehyphenate(lines, line_no)

    return "\n".join(lines)


def dehyphenate(lines: List[str], line_no: int) -> List[str]:
    next_line = lines[line_no + 1]
    word_suffix = next_line.split(" ")[0]

    lines[line_no] = lines[line_no][:-1] + word_suffix
    lines[line_no + 1] = lines[line_no + 1][len(word_suffix) :]
    return lines